In [32]:
#imports
#https://stackoverflow.com/questions/21402485/pandas-how-to-filter-a-df-to-get-unique-entries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
import time
import random
import sys
pd.set_option('display.max_colwidth', None)

from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from scipy import sparse
from matplotlib import pyplot as plt
import utils as ut

In [9]:
def rec_search(category, query, wout='no'):

    if category.lower() == 'video games':
        lookup, recommender = pd.read_pickle('./pickles/videog_look.pkl'),pd.read_pickle('./pickles/videog_rec.pkl')
    elif category.lower() == 'movies':
        lookup, recommender = pd.read_pickle('./pickles/movies_look.pkl'), pd.read_pickle('./pickles/movie_rec.pkl') 
    elif category.lower() == 'books':
        lookup, recommender = pd.read_pickle('./pickles/books_look.pkl'), pd.read_pickle('./pickles/books_rec.pkl')
    else:
        return "Sorry, that wasn't one of the available categories"

    try:
        #query=query.title() #uppercase first letter of each word in the query in case it's not entered that way
        query=query.lower() #lowercase
        titles = list(lookup[lookup['product_title'].str.contains(query)]['product_title'])
        q = titles[0] #this is the item to search for

        if wout.lower() == 'no':
            query_dict = dict(recommender.loc[q].sort_values())
            
            final_printout = ''
            for key in list(query_dict.keys())[1:11]:
                final_printout += '\n' #add new line
                final_printout += f'{key}' #print item name
                final_printout += '\n'
                final_printout += f"""
                This item has {round(lookup[lookup['product_title']==key]['tot_prod_reviews'].mean())} reviews 
                and a {round(lookup[lookup['product_title']==key]['avg_prod_stars'].mean(), 2)} average star rating"""
                final_printout += '\n'

            return print(f'''

            Recommending items similar to: {q}
            This item has {round(lookup[lookup['product_title']==q]['tot_prod_reviews'].mean())} reviews
            and a {round(lookup[lookup['product_title']==q]['avg_prod_stars'].mean(), 2)} average star rating.

            Here are the 10 recommended items for you based on your search parameters:
            
            {final_printout}
            
            ''')
            
        else:
            
            wout = wout.title() #capitlize first letters
            query_dict = dict(recommender.loc[q].sort_values())
            filtered_query = [] #make empty list
            for key, value in query_dict.items(): #index into dictionary of results
                if wout not in key: #check if avoided keyword is in results
                    filtered_query.append((key, value)) #make list of results that DON'T include "wout" keyword
            
            final_printout = ''
            for item in filtered_query[1:11]:
                final_printout += '\n' #add new line
                final_printout += f'{item[0]}' #print item name
                final_printout += '\n'
                final_printout += f"""
                This item has {round(lookup[lookup['product_title']==item[0]]['tot_prod_reviews'].mean())} reviews 
                and a {round(lookup[lookup['product_title']==item[0]]['avg_prod_stars'].mean(), 2)} average star rating"""
                final_printout += '\n'
            return print(f'''

            Recommending items similar to: {q}
            This item has {round(lookup[lookup['product_title']==q]['tot_prod_reviews'].mean())} reviews
            and a {round(lookup[lookup['product_title']==q]['avg_prod_stars'].mean(), 2)} average star rating.

            Here are the 10 recommended items for you based on your search EXCLUDING "{wout}":
            
            {final_printout}
            
            ''')
        
    except:
        return print(f'Sorry, "{query}" does not appear to be in the product database')

In [10]:
rec_search('books', 'batman')

Sorry, "batman" does not appear to be in the product database


Okay - this works and I got it running my flask app! What if, however, I return my output as a DATAFRAME instead of a long text segment? I think the table-ish formatting might look better and be easier to read.

In [2]:
#read in pickled dataframes
books_df, books_rec, movies_df, movies_rec, vg_df, vg_rec = pd.read_pickle('./pickles/books_look.pkl'), pd.read_pickle('./pickles/books_rec.pkl'), pd.read_pickle('./pickles/movies_look.pkl'), pd.read_pickle('./pickles/movie_rec.pkl'), pd.read_pickle('./pickles/videog_look.pkl'), pd.read_pickle('./pickles/videog_rec.pkl')

In [32]:
def rec_search(category, query, wout='no'):

    if category.lower() == 'video games':
        lookup, recommender = vg_df, vg_rec #pd.read_pickle('./pickles/videog_look.pkl'),pd.read_pickle('./pickles/videog_rec.pkl')
    elif category.lower() == 'movies':
        lookup, recommender = movies_df, movies_rec #pd.read_pickle('./pickles/movies_look.pkl'), pd.read_pickle('./pickles/movie_rec.pkl') 
    elif category.lower() == 'books':
        lookup, recommender = books_df, books_rec #pd.read_pickle('./pickles/books_look.pkl'), pd.read_pickle('./pickles/books_rec.pkl')
    else:
        return "Sorry, that wasn't one of the available categories"

    try:
        query=query.title() #uppercase first letter of each word in the query in case it's not entered that way
        titles = list(lookup[lookup['product_title'].str.contains(query)]['product_title'])
        q = titles[0] #this is the item to search for

        if wout.lower() == 'no':
            query_dict = dict(recommender.loc[q].sort_values())
            
            top10_prods = []
            num_prod_revs = []
            avg_prod_stars = []
            for key in list(query_dict.keys())[1:11]:
                top10_prods.append(key)
                num_prod_revs.append(round(lookup[lookup['product_title']==key]['tot_prod_reviews'].mean()))
                avg_prod_stars.append(round(lookup[lookup['product_title']==key]['avg_prod_stars'].mean(), 2))
            #print(top10_prods, num_prod_revs, avg_prod_stars)
            final_output_df = pd.DataFrame(data = {
                'Recommended Items':top10_prods,
                'Total Reviews for Product':num_prod_revs,
                'Avg Product Star Rating(1-5)':avg_prod_stars
            }, index=range(1,11))

            return final_output_df            
            
            
            
        else:
            
            wout = wout.title() #capitlize first letters
            query_dict = dict(recommender.loc[q].sort_values())
            filtered_query = [] #make empty list
            for key, value in query_dict.items(): #index into dictionary of results
                if wout not in key: #check if avoided keyword is in results
                    filtered_query.append((key, value)) #make list of results that DON'T include "wout" keyword
            
            top10_prods = []
            num_prod_revs = []
            avg_prod_stars = []
            for item in filtered_query[1:11]:
                top10_prods.append(item[0])
                num_prod_revs.append(round(lookup[lookup['product_title']==item[0]]['tot_prod_reviews'].mean()))
                avg_prod_stars.append(round(lookup[lookup['product_title']==item[0]]['avg_prod_stars'].mean(), 2))
            
                
            final_output_df = pd.DataFrame(data = {
                'Recommended Items':top10_prods,
                'Total Reviews for Product':num_prod_revs,
                'Avg Product Star Rating(1-5)':avg_prod_stars
                }, index=range(1,11))
                
                
                
            return final_output_df
        
    except:
        return f'Sorry, "{query}" does not appear to be in the product database'

In [33]:
rec_search('books', 'batman', 'batman')

,Recommended Items,Total Reviews for Product,Avg Product Star Rating(1-5)
1,V for Vendetta,92,4.54
2,"Daredevil Visionaries - Frank Miller, Vol. 1",18,4.28
3,Marvels,31,4.48
4,Frank Miller's Ronin,24,4.38
5,Infinity Gauntlet TPB,12,4.58
6,New X-Men Vol. 1: E is for Extinction (v. 1),27,4.26
7,Kingdom Come,134,4.37
8,Marvel 1602 HC (Marvel Heroes),16,4.25
9,Preacher VOL 06: War in the Sun (Preacher (DC Comics)),15,4.93
10,"6: Sandman, The: Fables & Reflections - Book VI",19,4.32


Okay, that'll do for a dataframe return item. I'm not convinced that will be the best return option but it's good to have the capability.

I've noticed my function is running slower than I'd like in app format (streamlit or flask). I want to find ways to trim the amount of work it's doing. I think I can cut some unused columns from the "lookup" dataframes. I can also potentially drop all but a single row for each product, since each row contains the aggregate information (tot_prod_reviews, avg_prod_stars) for each item. I'm not doing these calculations on the fly so I don't need to have all the contributing data, just a single representative.

In [11]:
lookup, recommender = pd.read_pickle('./pickles/videog_look.pkl'),pd.read_pickle('./pickles/videog_rec.pkl')

In [28]:
lookup.shape

(1648136, 7)

In [29]:
lookup['product_id'].nunique()

20952

In [30]:
lookup['product_title'].nunique()

15938

It's interesting that I have more unique product_id numbers than unique product_title(s). This indicates that some products were re-listed under different item numbers or possibly for sale under the same name by multiple sellers. I still think it makes sense to group these products together because, assuming reviews were based on the product not the seller, they will all reference the same item and ought to be grouped together. Indeed this is actually an advantage because anyone in the market for that product can see the aggregate of public opinion on the item rather than a fraction of it as represented only by buyers who purchased it from one unique seller.

In [33]:
lookup = lookup.groupby('product_title').first().copy()

This is a filter of my video-game dataframe that only shows one row per item. We can see that no lookup information is lost because the total product reviews and average stars are still accounted for in their respective columns. If I use this dataframe to lookup queries and return search information, not only would it occupy a much smaller portion of memory (about 16,000 rows compared to 1.6 million!), it would also save some of the list indexing / combining that I'm forced to do in my lookup function to account for all the duplicates I was getting back.

I can also drop customer_id, product_id, star_rating, and review_date because those do not apply to any returned values in my recommender function.

In [43]:
lookup.reset_index(inplace=True) #add index, was removed during groupby

In [35]:
lookup.drop(columns=['customer_id', 'product_id', 'star_rating', 'review_date'], inplace=True)

In [50]:
lookup.columns

Index(['product_title', 'tot_prod_reviews', 'avg_prod_stars'], dtype='object')

In [52]:
lookup.sort_values(by='tot_prod_reviews', ascending=False, inplace=True)

In [47]:
lookup['product_title'].map(lambda x: x.lower())

0                                                                              $100,000 pyramid - pc
1        (25) empty standard xbox 360 translucent green replacement games boxes / cases - vgbr14xbox
2                      (25) standard black nintendo ds empty replacement game cases boxes vgbr14dsbk
3                                          (4) wii fit balance board replacement foot leg extensions
4          (5) empty standard xbox 360 translucent green replacement games boxes / cases #dvbr14xbox
                                                    ...                                             
15933                                                    udraw marvel super hero squad: comic combat
15934                                                             udraw pictionary: ultimate edition
15935                                                    udraw studio: instant artist - nintendo wii
15936                     uxcell 3pcs replacement buttons conductive pads for nintendo nds 

In [53]:
query = 'batman'
titles = list(lookup[lookup['product_title'].map(lambda x: x.lower()).str.contains(query)]['product_title'])
titles[:5]

['Batman Arkham Origins',
 'Lego Batman 2 Super Hero',
 'Batman Arkham City',
 'Batman: Arkham City Game of the Year Edition',
 'Lego Batman']

In [55]:
lookup.reset_index(inplace=True) #reest index to reflect order by total product reviews
lookup.drop('index', inplace=True) #preview

,index,product_title,tot_prod_reviews,avg_prod_stars
0,9891,PlayStation 4 500GB Console [Old Model],10317,4.162256
1,4698,Grand Theft Auto V,8656,4.545055
2,1737,Call of Duty: Ghosts,7762,3.787426
3,1123,Battlefield 4,4795,3.666945
4,788,Assassin's Creed 4,4702,4.564866
...,...,...,...,...
15933,14210,Ultimate Civil War Battles: Robert E. Lee vs. Ulysses S. Grant - PC,11,1.181818
15934,3298,Duke Nukem: Critical Mass,11,3.454545
15935,3293,Duke Nukem,11,4.090909
15936,3284,DualPenSports - Nintendo 3DS,11,4.000000


In [56]:
#lookup.to_pickle('./pickles/vg_look_small.pkl')

Okay, that's the new video game lookup saved! The size of the old pickled df was 94 MB and the new one is just 1 MB so I expect to see significant returns in terms of speed searching the new one for the recommender. Now, I want to to apply the same treatment to movies and books.

In [70]:
def make_smaller_lookup(lookup_df):
    print(lookup_df.shape) #preview initial shape
    print(lookup_df['product_title'].nunique()) #unique product count
    lookup_df = lookup_df.groupby('product_title').first().copy() #only 1 row for each unique product
    lookup_df.reset_index(inplace=True) #add index, was removed during groupby
    lookup_df.drop(columns=['customer_id', 'product_id', 'star_rating', 'review_date'], inplace=True) #drop unused columns
    lookup_df.sort_values(by='tot_prod_reviews', ascending=False, inplace=True) #sort by most reviews
    lookup_df.reset_index(inplace=True) #reset index
    lookup_df.drop(columns=['index'], inplace=True) #drop original index (needed previously in order to sort)
    return lookup_df

Looks good! I'll save this in my python file and call it in at the top of my next recommender notebook.

In [71]:
lookup= pd.read_pickle('./pickles/movies_look.pkl')

In [72]:
movies_small = make_smaller_lookup(lookup)

(4405432, 7)
72385


In [73]:
movies_small #preview

,product_title,tot_prod_reviews,avg_prod_stars
0,Firefly: The Complete Series,4959,4.859649
1,Jillian Michaels - 30 Day Shred,4958,4.547802
2,Frozen,4569,4.688116
3,Frozen [Blu-ray],4492,4.701915
4,Mean Girls (Full Screen Edition),4409,4.993876
...,...,...,...
72380,Repeat Performance,11,3.363636
72381,Repentance,11,4.636364
72382,Replicant [Blu-ray],11,4.363636
72383,Glenn Gould: Hereafter,11,4.272727


In [ ]:
lookup= pd.read_pickle('./pickles/books_look.pkl')

In [ ]:
books_small = make_smaller_lookup(lookup)

In [ ]:
books_small #preview

In [ ]:
lookup= pd.read_pickle('./pickles/videog_look.pkl')

In [ ]:
vg_small = make_smaller_lookup(lookup)

In [ ]:
vg_small #preview